# Proyecto 2. Analisis Exploratorio

## Deteccion de covid en radiografias de torax

### Integrantes
#### Cristina Bautista 161260
#### Jose Block 18935
#### Esteban Cabrera 17781
#### Byron Mota 15246

Primero correr esta dependecia

In [ ]:
!pip3 install python-gdcm
import gdcm

Resetear y volver a correr

In [ ]:
import os
import pandas as pd
import numpy as np
!pip install --upgrade numpy
import matplotlib.pyplot as plt
import matplotlib
import pydicom as dicom
import cv2
import ast

import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.listdir('/kaggle/input/siim-covid19-detection/')

In [ ]:
df1 = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv')
df2 = pd.read_csv('/kaggle/input/siim-covid19-detection/train_study_level.csv')

In [ ]:
df1['id_dcm'] = df1['id']
df1['id_dcm'] = df1['id'].str.replace('_image', '.dcm')
df1['id'] = df1['id'].str.replace('_image', '')
df2['id'] = df2['id'].str.replace('_study', '')

In [ ]:
df1.head()

In [ ]:
df1.info()

In [ ]:
df2.head()

In [ ]:
df2.info()

In [ ]:
df = pd.merge(df1, df2, left_on='StudyInstanceUID', right_on='id', how='inner')

Se uso de guia para leer los archivos .dcm del repositorio: https://www.kaggle.com/drcapa/siim-fisabio-rsna-covid-19-detection-starter

In [ ]:
len(df)

In [ ]:
path = '/kaggle/input/siim-covid19-detection/'

In [ ]:
temp = df.loc[0, 'StudyInstanceUID']
temp

In [ ]:
temp_depth2 = os.listdir(path+'train/'+temp)
temp_depth2

In [ ]:
temp_train_path = path+'train/'+temp+'/'+temp_depth2[0]
temp_train_path

In [ ]:
os.listdir(temp_train_path)

In [ ]:
def extraction(i):
    data_file = dicom.dcmread(complete_path_train)
    img = data_file.pixel_array
    return img

In [ ]:
def extractionPath(i):
    path_train = path + 'train/' + df.loc[i, 'StudyInstanceUID']
    last_folder_in_path = os.listdir(path_train)[0]
    path_train = path_train + '/{}/'.format(last_folder_in_path)
    img_id = df.loc[i, 'id_dcm']
    complete_path_train = path_train + img_id
    return complete_path_train

In [ ]:
img_paths = []
for i in range(len(df)):
     img_paths.append(extractionPath(i))

In [ ]:
type(img_paths)

In [ ]:
df['Image_Path'] = img_paths

In [ ]:
df

In [ ]:
df[df.eq('65761e66de9f').any(1)]

In [ ]:
paths = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))

In [ ]:
path = [x for x in paths if "test" not in x and 'csv' not in x]
path[:10]

In [ ]:
order_paths = []
for i in df['id_dcm']:
    for j in path:
        if i == j[-16:-1]:
            order_paths.append(j)

In [ ]:
order_paths

In [ ]:
import cv2
import time
def extract_resized_and_origin_img_info(path_list):
    img_list = []
    origin_img_heights = []
    origin_img_widths = []
    i = 0
    for path in path_list:
        data_file = dicom.read_file(path)
        img = data_file.pixel_array

            
        origin_img_heights.append(img.shape[0])
        origin_img_widths.append(img.shape[1])

        
        # scailing to 0~255
        img = (img - np.min(img)) / np.max(img)
        img = (img * 255).astype(np.uint8)
        
        # resizing to 4000+ to 255 default
        img = cv2.resize(img, (255,255))
        img_list.append(img)
        img_array = np.array(img_list)
        i += 1
        if i % 100 == 0:
            print('{} / {}'.format(len(img_array),len(path_list)))
            time.sleep(2)
    return img_array, origin_img_heights, origin_img_widths

In [ ]:
test_imgs_new, origin_img_heights2, origin_img_widths2 = extract_resized_and_origin_img_info(path[:])

In [ ]:
type(test_imgs_new)

In [ ]:
test_imgs_new = np.array(test_imgs_new)

In [ ]:
test_imgs_new.shape

In [ ]:
test_imgs_new_4dim = test_imgs_new[0:1,:,:,np.newaxis]
test_imgs_new_4dim.shape

In [ ]:
x_scale_list=[]
y_scale_list=[]
if len(origin_img_heights2) == len(origin_img_widths2):
    for i in range(len(origin_img_heights2)):
        x_scale = 255 / origin_img_widths2[i]
        x_scale_list.append(x_scale)
        print(i)
        y_scale = 255 / origin_img_heights2[i]
        y_scale_list.append(y_scale)

In [ ]:
clasificadores = list(df.columns[6:10])
clasificadores

In [ ]:
!mkdir ./genData
!mkdir ./genData/NegPeu
!mkdir ./genData/Typical
!mkdir ./genData/Indeterminate
!mkdir ./genData/Atypical

In [ ]:
imgs_NegPeu = list(df[df[clasificadores[0]]==1].index)
for idx in imgs_NegPeu:
    plt.imsave('./genData/NegPeu/{}.jpg'.format(df.loc[idx,'id_x']), test_imgs_new[idx], cmap='gray')

In [ ]:
imgs_Typical = list(df[df[clasificadores[1]]==1].index)
for idx in imgs_Typical:
    plt.imsave('./genData/Typical/{}.jpg'.format(df.loc[idx,'id_x']), test_imgs_new[idx], cmap='gray')

In [ ]:
imgs_Indeterminate = list(df[df[clasificadores[2]]==1].index)
for idx in imgs_Indeterminate:
    plt.imsave('./genData/Indeterminate/{}.jpg'.format(df.loc[idx,'id_x']), test_imgs_new[idx], cmap='gray')

In [ ]:
imgs_Atypical = list(df[df[clasificadores[3]]==1].index)
for idx in imgs_Atypical:
    plt.imsave('./genData/Atypical/{}.jpg'.format(df.loc[idx,'id_x']), train_imgs[idx], cmap='gray')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
idatagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=3,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    horizontal_flip=False,
    fill_mode='reflect',
    validation_split=0.2
)

In [ ]:
train_gen = idatagen.flow_from_directory(
    './genData',
    batch_size=64,
    target_size=(256, 256),
    class_mode='categorical',
    color_mode='grayscale',
    subset = 'training'
)

valid_gen = idatagen.flow_from_directory(
    './genData',
    batch_size = 64,
    target_size = (256, 256),
    class_mode = 'categorical',
    color_mode='grayscale',
    subset = 'validation'
)

## Clasificador Básico

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(256, 256,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])
model.summary() 

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
checkpoint = ModelCheckpoint(
    filepath = './checkpoint1.ckpt',
    save_weights_only = True,
    save_best_only = True,
    monitor = 'val_loss',
    verbose=1
)

In [ ]:
model.fit(
    train_gen,
    validation_data = (valid_gen),
    epochs = 20,
    callbacks=[checkpoint]
)

In [ ]:
model.load_weights('./checkpoint1.ckpt')

In [ ]:
model.evaluate(valid_gen)

In [ ]:
model.save('./baseCnn.h5')

In [ ]:
model.predict(test_imgs_new_4dim)

In [ ]:
a = input('Ingrese el directorio de una imagen')

In [ ]:
a_4dim = a[:,:,:,np.newaxis]
a_4dim.shape

In [ ]:
model.predict(a_4dim)